In [4]:
import tensorflow as tf
from tensorflow import keras

In [2]:
import os
root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
    
run_logdir = get_run_logdir()

In [5]:
# A classification MLP with two hidden layers.

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")                   
])

C:\Users\arnol\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [7]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full),(X_test, y_test) = fashion_mnist.load_data()

In [8]:
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [9]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
history = model.fit(X_train, y_train, epochs=30, 
                    validation_data=(X_valid, y_valid),
                    callbacks=[tensorboard_cb])

Epoch 1/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.6694 - loss: 1.0378 - val_accuracy: 0.8336 - val_loss: 0.4953
Epoch 2/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8263 - loss: 0.4996 - val_accuracy: 0.8398 - val_loss: 0.4578
Epoch 3/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8421 - loss: 0.4526 - val_accuracy: 0.8474 - val_loss: 0.4356
Epoch 4/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8526 - loss: 0.4217 - val_accuracy: 0.8630 - val_loss: 0.3961
Epoch 5/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8608 - loss: 0.3945 - val_accuracy: 0.8680 - val_loss: 0.3778
Epoch 6/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8644 - loss: 0.3788 - val_accuracy: 0.8700 - val_loss: 0.3725
Epoch 7/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8720 - loss: 0.3662 - val_accuracy: 0.8636 - val_loss: 0.3759
Epoch 8/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8752 - loss: 0.3558 -

In [10]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006

In [13]:
# More visualizations in tensorboard, 

import numpy as np

test_logdir = get_run_logdir()
writer = tf.summary.create_file_writer(test_logdir)
with writer.as_default():
 for step in range(1, 1000 + 1):
     tf.summary.scalar("my_scalar", np.sin(step / 10), step=step)
     data = (np.random.randn(100) + 2) * step / 100 # some random data
     tf.summary.histogram("my_hist", data, buckets=50, step=step)
     images = np.random.rand(2, 32, 32, 3) # random 32×32 RGB images
     tf.summary.image("my_images", images * step / 1000, step=step)
     texts = ["The step is " + str(step), "Its square is " + str(step**2)]
     tf.summary.text("my_text", texts, step=step)
     sine_wave = tf.math.sin(tf.range(12000) / 48000 * 2 * np.pi * step)
     audio = tf.reshape(tf.cast(sine_wave, tf.float32), [1, -1, 1])
     tf.summary.audio("my_audio", audio, sample_rate=48000, step=step)